In [21]:

import numpy as np
import pandas as pd

import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
# Set the desired float format 
pd.options.display.float_format = '{:.2f}'.format

In [3]:
yellow_taxi_data=pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')

In [4]:
yellow_taxi_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.00,1.72,1.00,N,186,79,2,17.70,1.00,0.50,0.00,0.00,1.00,22.70,2.50,0.00
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.00,1.80,1.00,N,140,236,1,10.00,3.50,0.50,3.75,0.00,1.00,18.75,2.50,0.00
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.00,4.70,1.00,N,236,79,1,23.30,3.50,0.50,3.00,0.00,1.00,31.30,2.50,0.00
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.00,1.40,1.00,N,79,211,1,10.00,3.50,0.50,2.00,0.00,1.00,17.00,2.50,0.00
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.00,0.80,1.00,N,211,148,1,7.90,3.50,0.50,3.20,0.00,1.00,16.10,2.50,0.00


In [5]:
yellow_taxi_data.shape

(2964624, 19)

In [6]:
yellow_taxi_data['duration'] = yellow_taxi_data.tpep_dropoff_datetime - yellow_taxi_data.tpep_pickup_datetime
print(yellow_taxi_data.duration.describe())

count                      2964624
mean     0 days 00:15:36.777037155
std      0 days 00:34:51.063215532
min              -1 days +23:46:26
25%                0 days 00:07:09
50%                0 days 00:11:38
75%                0 days 00:18:41
max                6 days 13:35:24
Name: duration, dtype: object


In [7]:
yellow_taxi_data.duration = yellow_taxi_data.duration.dt.total_seconds() / 60
print(yellow_taxi_data.duration.describe())

count   2964624.00
mean         15.61
std          34.85
min         -13.57
25%           7.15
50%          11.63
75%          18.68
max        9455.40
Name: duration, dtype: float64


In [8]:
max(yellow_taxi_data.duration)

9455.4

In [9]:
clean_yellow_taxi_data=yellow_taxi_data[(yellow_taxi_data.duration>=1) & (yellow_taxi_data.duration<=60)]
clean_yellow_taxi_data

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.00,1.72,1.00,N,186,79,2,17.70,1.00,0.50,0.00,0.00,1.00,22.70,2.50,0.00,19.80
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.00,1.80,1.00,N,140,236,1,10.00,3.50,0.50,3.75,0.00,1.00,18.75,2.50,0.00,6.60
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.00,4.70,1.00,N,236,79,1,23.30,3.50,0.50,3.00,0.00,1.00,31.30,2.50,0.00,17.92
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.00,1.40,1.00,N,79,211,1,10.00,3.50,0.50,2.00,0.00,1.00,17.00,2.50,0.00,8.30
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.00,0.80,1.00,N,211,148,1,7.90,3.50,0.50,3.20,0.00,1.00,16.10,2.50,0.00,6.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-31 23:45:59,2024-01-31 23:54:36,NaN,3.18,NaN,None,107,263,0,15.77,0.00,0.50,2.00,0.00,1.00,21.77,NaN,NaN,8.62
2964620,1,2024-01-31 23:13:07,2024-01-31 23:27:52,NaN,4.00,NaN,None,114,236,0,18.40,1.00,0.50,2.34,0.00,1.00,25.74,NaN,NaN,14.75
2964621,2,2024-01-31 23:19:00,2024-01-31 23:38:00,NaN,3.33,NaN,None,211,25,0,19.97,0.00,0.50,0.00,0.00,1.00,23.97,NaN,NaN,19.00
2964622,2,2024-01-31 23:07:23,2024-01-31 23:25:14,NaN,3.06,NaN,None,107,13,0,23.88,0.00,0.50,5.58,0.00,1.00,33.46,NaN,NaN,17.85


In [10]:
left_data=(clean_yellow_taxi_data.shape[0]/yellow_taxi_data.shape[0])*100
left_data

97.78326020432945

In [11]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df['duration'].dt.total_seconds() / 60
    df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].fillna(-1)
    df[categorical] = df[categorical].astype(str)
    
    return df

In [12]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet')

In [13]:
df_train

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.00,1.72,1.00,N,186,79,2,17.70,1.00,0.50,0.00,0.00,1.00,22.70,2.50,0.00,19.80
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.00,1.80,1.00,N,140,236,1,10.00,3.50,0.50,3.75,0.00,1.00,18.75,2.50,0.00,6.60
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.00,4.70,1.00,N,236,79,1,23.30,3.50,0.50,3.00,0.00,1.00,31.30,2.50,0.00,17.92
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.00,1.40,1.00,N,79,211,1,10.00,3.50,0.50,2.00,0.00,1.00,17.00,2.50,0.00,8.30
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.00,0.80,1.00,N,211,148,1,7.90,3.50,0.50,3.20,0.00,1.00,16.10,2.50,0.00,6.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-31 23:45:59,2024-01-31 23:54:36,NaN,3.18,NaN,None,107,263,0,15.77,0.00,0.50,2.00,0.00,1.00,21.77,NaN,NaN,8.62
2964620,1,2024-01-31 23:13:07,2024-01-31 23:27:52,NaN,4.00,NaN,None,114,236,0,18.40,1.00,0.50,2.34,0.00,1.00,25.74,NaN,NaN,14.75
2964621,2,2024-01-31 23:19:00,2024-01-31 23:38:00,NaN,3.33,NaN,None,211,25,0,19.97,0.00,0.50,0.00,0.00,1.00,23.97,NaN,NaN,19.00
2964622,2,2024-01-31 23:07:23,2024-01-31 23:25:14,NaN,3.06,NaN,None,107,13,0,23.88,0.00,0.50,5.58,0.00,1.00,33.46,NaN,NaN,17.85


In [14]:
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet')

In [15]:
df_val

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,2,2024-02-01 00:04:45,2024-02-01 00:19:58,1.00,4.39,1.00,N,68,236,1,20.50,1.00,0.50,1.28,0.00,1.00,26.78,2.50,0.00,15.22
1,2,2024-02-01 00:56:31,2024-02-01 01:10:53,1.00,7.71,1.00,N,48,243,1,31.00,1.00,0.50,9.00,0.00,1.00,45.00,2.50,0.00,14.37
2,2,2024-02-01 00:07:50,2024-02-01 00:43:12,2.00,28.69,2.00,N,132,261,2,70.00,0.00,0.50,0.00,6.94,1.00,82.69,2.50,1.75,35.37
3,1,2024-02-01 00:01:49,2024-02-01 00:10:47,1.00,1.10,1.00,N,161,163,1,9.30,3.50,0.50,2.85,0.00,1.00,17.15,2.50,0.00,8.97
4,1,2024-02-01 00:37:35,2024-02-01 00:51:15,1.00,2.60,1.00,N,246,79,2,15.60,3.50,0.50,0.00,0.00,1.00,20.60,2.50,0.00,13.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3007521,2,2024-02-29 23:20:17,2024-02-29 23:28:25,NaN,0.35,NaN,None,164,170,0,21.02,0.00,0.50,0.00,0.00,1.00,25.02,NaN,NaN,8.13
3007522,2,2024-02-29 23:38:37,2024-02-29 23:49:37,NaN,1.99,NaN,None,249,45,0,18.78,0.00,0.50,0.00,0.00,1.00,22.78,NaN,NaN,11.00
3007523,2,2024-02-29 23:14:40,2024-02-29 23:19:29,NaN,0.84,NaN,None,113,249,0,8.82,0.00,0.50,0.00,0.00,1.00,12.82,NaN,NaN,4.82
3007524,1,2024-02-29 23:12:06,2024-02-29 23:21:53,NaN,0.00,NaN,None,164,107,0,14.37,0.00,0.50,0.00,0.00,1.00,18.37,NaN,NaN,9.78


In [16]:
categorical = ['PULocationID', 'DOLocationID'] 

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)


In [17]:
X_train

<2898906x518 sparse matrix of type '<class 'numpy.float64'>'
	with 5797812 stored elements in Compressed Sparse Row format>

In [18]:
X_train.shape

(2898906, 518)

In [19]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [22]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [23]:
print(mean_squared_error(y_train, y_pred, squared=False))

7.946173359562653


In [24]:
y_pred = lr.predict(X_val)

print(mean_squared_error(y_val, y_pred, squared=False))

8.123382950127723
